In [1]:
# Libraries
import pandas as pd
import numpy as np

In [2]:
# Global data import for all viz's
raw = pd.read_csv("https://github.com/zmosDS/recession_impact_analysis/releases/download/1/bls_employment_stats.csv")

raw.head()

,series_id,industry_name,supersector_name,naics_code,year,month,value,seasonal,month_num
0,CES1000000001,Mining and logging,Mining and logging,-,1995,January,651.0,S,1
1,CES1000000001,Mining and logging,Mining and logging,-,1995,February,647.0,S,2
2,CES1000000001,Mining and logging,Mining and logging,-,1995,March,646.0,S,3
3,CES1000000001,Mining and logging,Mining and logging,-,1995,April,644.0,S,4
4,CES1000000001,Mining and logging,Mining and logging,-,1995,May,641.0,S,5


In [3]:
# Filtering for YoY change (Visualization 1)

# Filter for total nonfarm
nonfarm = raw[raw["industry_name"] == "Total nonfarm"].copy()

# Compute annual averages
annual = nonfarm.groupby("year")["value"].mean().reset_index()

# Compute YoY percent change
annual["yoy_change"] = annual["value"].pct_change() * 100

# Save
annual.to_csv("viz1_yoy_change.csv", index=False)
print(f"Csv Saved")

Csv Saved


In [4]:
# Create a global timeline (t_global) for time-based filtering
raw["month_num"] = raw["month_num"].astype(str).str.zfill(2)
raw["month_num"] = pd.to_numeric(raw["month_num"], errors="coerce")
raw["t_global"] = (raw["year"].astype(int) * 12) + raw["month_num"]

# Filter to:
# 1. Seasonally adjusted data only (S), removes seasonal noise
# 2. Supersector aggregates only (industry_name matches supersector_name)
# 3. Relevant columns
viz34 = raw[(raw["seasonal"] == "S") & 
            (raw["industry_name"].str.lower() == raw["supersector_name"].str.lower())
            ][["supersector_name", "year", "month_num", "month", "value", "t_global"]].copy()

# Format date columns
viz34["date"] = pd.to_datetime(viz34["year"].astype(str) + "-" + 
                               viz34["month_num"].astype(str).str.zfill(2) + "-01"
                               ).dt.strftime("%Y-%m-%d")

# Sort and save
viz34 = viz34.sort_values(["supersector_name", "year", "month_num"])
viz34.to_csv("viz3_and4.csv", index=False)

print(f"Filtered Rows: {len(viz34):,}")
print(f"File Size: {viz34.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"Csv Saved")

Filtered Rows: 5,520
File Size: 1.17 MB
Csv Saved
